In [1]:
!pip install selenium

In [2]:
headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [19]:
import time
import pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from selenium import webdriver

# Selenium setup
driver = webdriver.Chrome()
driver.get("https://www.zomato.com/delhi/delivery")
time.sleep(2)

scroll_pause_time = 3
screen_height = driver.execute_script("return window.screen.height;")
i = 1

while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)

    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if (screen_height) * i > scroll_height:
        break

soup = BeautifulSoup(driver.page_source, "html.parser")
divs1 = soup.findAll('div', class_='sc-1mo3ldo-0')
divs = divs1[4:]

urls = []
rest_name = []
ratings = []
#addresses = []
price = []
cuisine = []
data = pd.DataFrame()


for parent in divs:
    try:
        isCom = True
        try:
            if "Inspiration for your first order" in parent.find("h3").text:
                print("no need first")
                isCom = False
        except Exception as e:
            pass

        try:
            if "Best Food in" in parent.find("h1").text:
                print("no need best")
                isCom = False
        except Exception as e:
            pass

        try:
            if "Top brands for" in parent.find("h3").text:
                print("no need top")
                isCom = False
        except Exception as e:
            pass

        try:
            if "End of search results" in parent.find("h3").text:
                print("no need end")
                isCom = False
                break
        except Exception as e:
            pass

        if isCom:
            divas = parent.findAll('div', class_='jumbo-tracker')
            
            

            for par in divas:
                link_tag = par.find("a")
                base = "https://www.zomato.com"

                if 'href' in link_tag.attrs:
                    link = link_tag.get('href')

                url = urljoin(base, link)
                urls.append(url)

                rating_tag = par.div.a.next_sibling.div.div.div.div.div.div.div.text
                price_tag = par.div.a.next_sibling.p.next_sibling.text
                cuisine_tag = par.div.a.next_sibling.p.text

                ratings.append(rating_tag)
                price.append(price_tag)
                cuisine.append(cuisine_tag)

                driver.get(url)
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source, "html.parser")

                rest_namea = [soup.select('h1.sc-7kepeu-0')[-1].text]
                rest_name.extend(rest_namea)

                cuisine_name = [h.text for h in soup.find_all('h4', class_="sc-1s0saks-15 iSmBPS")]
                priceL = [h1.text for h1 in soup.find_all('span', class_="sc-17hyc2s-1 cCiQWA")]
                #description = [h2.text for h2 in soup.find_all('p', class_="sc-1s0saks-12 hcROsL")]
                
                if len(rest_namea) == len(cuisine_name) == len(priceL) == len(description):
                    data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL))
                    data = data.append(data1, ignore_index=True)


                if data.empty:

                    data = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL))
                else:
                    data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL))
                    data = pd.concat([data, data1], ignore_index=True)


                data.to_csv('food_menu_delhi.csv', index=False)

                #addresses.append(soup.findAll('a', class_='sc-cmTdod')[0].text)


    except Exception as e:
        print(e)
        pass

out_df2 = pd.DataFrame(
    {'links': urls, 'names': rest_name, 'ratings': ratings, 'price for two': price,
     'cuisine': cuisine})

out_df2.to_csv("delhi.csv")
driver.close()

no need first
no need top
no need end


In [21]:
pd.read_csv('Blr.csv')

,Unnamed: 0,links,names,ratings,price for two,cuisine
0,0,https://www.zomato.com/bangalore/empire-restau...,Empire Restaurant,4.1,₹100 for one,"North Indian, Biryani, Kebab, South Indian, Mu..."
1,1,https://www.zomato.com/bangalore/truffles-2-st...,Truffles,4.3,₹100 for one,"Burger, Italian, Desserts, Continental, Americ..."
2,2,https://www.zomato.com/bangalore/nandhana-pala...,Nandhana Palace,4.1,₹100 for one,"Andhra, Biryani, North Indian, Chinese, Desser..."
3,3,https://www.zomato.com/bangalore/leons-burgers...,Leon's Burgers & Wings,4.2,₹100 for one,"Burger, Fast Food, Wraps"
4,4,https://www.zomato.com/bangalore/the-only-plac...,The Only Place,4.4,₹100 for one,"Italian, Steak, Continental, Fast Food, Desser..."
...,...,...,...,...,...,...
911,911,https://www.zomato.com/bangalore/suchalis-arti...,Suchali's Artisan Bakehouse,4.0,₹250 for one,"Bakery, Desserts, Beverages"
912,912,https://www.zomato.com/bangalore/cottage-loaf-...,Cottage Loaf - Koramangala,4.5,₹100 for one,"Desserts, Cafe"
913,913,https://www.zomato.com/bangalore/mumbai-masala...,Mumbai Masala,3.9,₹100 for one,"Street Food, Fast Food"
914,914,https://www.zomato.com/bangalore/shero-home-fo...,Shero Home Food - North Indian,4.0,₹150 for one,"North Indian, Healthy Food, Desserts"


In [ ]:
# import time
# import pandas as pd
# from urllib.parse import urljoin
# from bs4 import BeautifulSoup
# from selenium import webdriver

# # Selenium setup
# driver = webdriver.Chrome()
# driver.get("https://www.zomato.com/bangalore/delivery")
# time.sleep(2)

# scroll_pause_time = 3
# screen_height = driver.execute_script("return window.screen.height;")
# i = 1

# while True:
#     driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
#     i += 1
#     time.sleep(scroll_pause_time)

#     scroll_height = driver.execute_script("return document.body.scrollHeight;")
#     if (screen_height) * i > scroll_height:
#         break

# soup = BeautifulSoup(driver.page_source, "html.parser")
# divs1 = soup.findAll('div', class_='sc-1mo3ldo-0')
# divs = divs1[4:]

# urls = []
# rest_name = []
# ratings = []
# addresses = []
# price = []
# cuisine = []
# data = pd.DataFrame()

# for parent in divs:
#     try:
#         isCom = True
#         try:
#             if "Inspiration for your first order" in parent.find("h3").text:
#                 print("no need first")
#                 isCom = False
#         except Exception as e:
#             pass

#         try:
#             if "Best Food in" in parent.find("h1").text:
#                 print("no need best")
#                 isCom = False
#         except Exception as e:
#             pass

#         try:
#             if "Top brands for" in parent.find("h3").text:
#                 print("no need top")
#                 isCom = False
#         except Exception as e:
#             pass

#         try:
#             if "End of search results" in parent.find("h3").text:
#                 print("no need end")
#                 isCom = False
#                 break
#         except Exception as e:
#             pass

#         if isCom:
#             divas = parent.findAll('div', class_='jumbo-tracker')
            
#             for par in divas:
#                 link_tag = par.find("a")
#                 base = "https://www.zomato.com"

#                 if 'href' in link_tag.attrs:
#                     link = link_tag.get('href')

#                 url = urljoin(base, link)
#                 urls.append(url)

#                 rating_tag = par.div.a.next_sibling.div.div.div.div.div.div.div.text
#                 price_tag = par.div.a.next_sibling.p.next_sibling.text
#                 cuisine_tag = par.div.a.next_sibling.p.text

#                 ratings.append(rating_tag)
#                 price.append(price_tag)
#                 cuisine.append(cuisine_tag)

#                 driver.get(url)
#                 time.sleep(3)
#                 soup = BeautifulSoup(driver.page_source, "html.parser")

#                 rest_namea = [soup.select('h1.sc-7kepeu-0')[-1].text]
#                 rest_name.extend(rest_namea)

#                 cuisine_name = [h.text for h in soup.find_all('h4', class_="sc-1s0saks-15 iSmBPS")]
#                 priceL = [h1.text for h1 in soup.find_all('span', class_="sc-17hyc2s-1 cCiQWA")]
#                 description = [h2.text for h2 in soup.find_all('p', class_="sc-1s0saks-12 hcROsL")]
                
#                 if len(rest_namea) == len(cuisine_name) == len(priceL) == len(description):
#                     data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
#                     data = data.append(data1, ignore_index=True)


#                 if data.empty:
#                     data = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
#                 else:
#                     data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
#                     data = data.append(data1, ignore_index=True)

#                 data.to_csv('food_menu.csv', index=False)

#                 addresses.append(soup.findAll('a', class_='sc-cmTdod')[0].text)

#     except Exception as e:
#         print(e)
#         pass

# # Check if all lists have the same length
# if len(urls) == len(rest_name) == len(ratings) == len(addresses) == len(price) == len(cuisine):
#     out_df2 = pd.DataFrame({
#         'links': urls, 
#         'names': rest_name,  
#         'ratings': ratings, 
#         'price for two': price,
#         'cuisine': cuisine
#     })

#     out_df2.to_csv("restaurants_Banglore.csv", index=False)
# else:
#     print("Lists have different lengths. Data inconsistency detected.")

# driver.close()


In [ ]:
print(len(ratings))
